In [8]:
from ortools.linear_solver import pywraplp
import pandas as pd

# Constants
num_people = 6
num_shifts = 3  # Morning, afternoon, evening
num_days = 365 + ((num_people - 1) * 8)  # Adding predating days
shift_pattern = [0, 0, 1, 1, 2, 2] + [-1, -1, -1, -1]  # Morning, morning, afternoon, afternoon, evening, evening, off, off, off, off
pattern_len = len(shift_pattern)
shift_names = ["Morning", "Afternoon", "Evening"]

# Initialize the MIP solver with the CBC backend.
solver = pywraplp.Solver.CreateSolver('CBC')

# Decision variables
# X[i][j][k] = 1 means person i works on day j during shift k
X = [[[solver.IntVar(0, 1, f"person_{i}_day_{j}_shift_{k}")
       for k in range(num_shifts)]
      for j in range(num_days)]
     for i in range(num_people)]

# Constraints
# Each shift on each day is assigned to exactly one person
for j in range(num_days):
    for k in range(num_shifts):
        solver.Add(solver.Sum(X[i][j][k] for i in range(num_people)) == 1)

# Each person follows the shift pattern
for i in range(num_people):
    for j in range(num_days - pattern_len):
        for k in range(pattern_len):
            if shift_pattern[k] != -1:
                solver.Add(X[i][j+k][shift_pattern[k]] == 1)
            else:
                solver.Add(solver.Sum(X[i][j+k][s] for s in range(num_shifts)) == 0)

# Solve the problem
status = solver.Solve()
print ("Status:", status)

# Check if a solution was found
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    # Create a DataFrame to store the schedule
    roster = pd.DataFrame()

    for i in range(num_people):
        for j in range(num_days):
            for k in range(num_shifts):
                if X[i][j][k].solution_value() == 1:
                    # Only include days after the predating period
                    if j >= (num_people - 1) * 8:
                        roster = roster.append({"Person": i+1,
                                                "Day": j - ((num_people - 1) * 8) + 1,
                                                "Shift": shift_names[k]}, ignore_index=True)

    # Sort the roster by day and shift
    roster.sort_values(by=["Day", "Shift"], inplace=True)
    roster.reset_index(drop=True, inplace=True)
else:
    roster = "No solution found."

print(roster)


No solution found.
